In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate, interpolate
from src.synthetic import ODEModelBaseClass, gillespie_noise_raw

Define classes & functions.  Architecture based on `synthetic.py`, equations based on Rocsoreanu et al. (2000)

In [ ]:
class FitzHughNagumoModel(ODEModelBaseClass):
    # Defined according to Rocsoreanu et al. (2000), eqn 1.1.17
    # VARIABLES
    # x :: membrane voltage
    # y :: linear recovery variable
    # PARAMETERS
    # a ::
    # b ::
    # c ::
    def odes(self, var_array, timeaxis, *args):
        # Define variables
        x = var_array[0]
        y = var_array[1]
        # Define parameters
        a = args[0]
        b = args[1]
        c = args[2]
        # Define differential equations
        dx_dt = c*(x + y - (x**3)/3)
        dy_dt = (-1/c)*(x - a + b*y)
        return [dx_dt, dy_dt]
    
class FitzHughNagumoModelStochastic(FitzHughNagumoModel):
    def __init__(self, timeaxis, init_cond, ode_parameters):
        super().__init__(timeaxis, init_cond, ode_parameters)

        self.ode_parameters_original = self.ode_parameters

        # a
        self.a_array = self.ode_parameters_original["a"]
        self.a_interp1d = interpolate.interp1d(self.timeaxis, self.a_array)
        self.ode_parameters["a"] = self.a

        # b
        self.b_array = self.ode_parameters_original["b"]
        self.b_interp1d = interpolate.interp1d(self.timeaxis, self.b_array)
        self.ode_parameters["b"] = self.b
        
        # c
        self.c_array = self.ode_parameters_original["c"]
        self.c_interp1d = interpolate.interp1d(self.timeaxis, self.c_array)
        self.ode_parameters["c"] = self.c

    # Assumes timeaxis and ext_stimulus_array have the same number of elements
    def a(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.a_interp1d(np.asarray(t))

    def b(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.b_interp1d(np.asarray(t))
    
    def c(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.c_interp1d(np.asarray(t))

    def odes(self, var_array, t, *args):
        # Define variables
        x = var_array[0]
        y = var_array[1]
        # Define parameters
        a = args[0]
        b = args[1]
        c = args[2]
        # Define differential equations
        dx_dt = c(t)*(x + y - (x**3)/3)
        dy_dt = (-1/c(t))*(x - a(t) + b(t)*y)
        return [dx_dt, dy_dt]

    
def fitzhugh_nagumo(
    timeaxis=np.linspace(0, 1000, 1000),
    voltage_init=0,
    recovery_init=0,
    a=1,
    b=1,
    c=1,
):
    model = FitzHughNagumoModel(
        timeaxis=timeaxis,
        init_cond={"x": voltage_init, "y": recovery_init},
        ode_parameters={"a": a, "b": b, "c": c},
    )
    var_out = model.solver()
    voltage = var_out.T[0]
    recovery = var_out.T[1]
    return voltage, recovery

def fitzhugh_nagumo_stochastic(
    timeaxis=np.linspace(0, 1000, 1000),
    voltage_init=0,
    recovery_init=0,
    a_array=0.05 * np.random.rand(1000) + 1,
    b_array=0.05 * np.random.rand(1000) + 1,
    c_array=0.05 * np.random.rand(1000) + 1,
):
    model = FitzHughNagumoModelStochastic(
        timeaxis=timeaxis,
        init_cond={"x": voltage_init, "y": recovery_init},
        ode_parameters={
            "a": a_array,
            "b": b_array,
            "c": c_array,
        },
    )
    var_out = model.solver()
    voltage = var_out.T[0]
    recovery = var_out.T[1]
    return voltage, recovery

Deterministic

In [ ]:
v, r = fitzhugh_nagumo(
    timeaxis=np.linspace(0, 200, 200),
    a=1e-10, b=1.471143170154, c=5)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(v)

Stochastic - white noise

In [ ]:
a_array=0.002 * np.random.rand(200) + 0.994
b_array=0.00 * np.random.rand(200) + 0
c_array=0.00 * np.random.rand(200) + 5

vs, rs = fitzhugh_nagumo_stochastic(
    timeaxis=np.linspace(0, 200, 200),
    a_array=a_array, b_array=b_array, c_array=c_array
)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(vs)

plt.subplots(figsize=(20,5))
plt.plot(a_array)

In [ ]:
# b

a_array=0.00 * np.random.rand(200) + 1e-10
b_array=0.015 * np.random.rand(200) + 1.46
c_array=0.00 * np.random.rand(200) + 5

vs, rs = fitzhugh_nagumo_stochastic(
    timeaxis=np.linspace(0, 200, 200),
    a_array=a_array, b_array=b_array, c_array=c_array
)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(vs)

plt.subplots(figsize=(20,5))
plt.plot(b_array)

> Conclusion: As expected, if the stochastic parameter is near a sensitive point and varies a bit, the system alternates between several types of oscillator.  It doesn't lead to as much of a noise in the time series as I expected, but I've come to expect that by now.  The problem now is surveying the parameter space to find other parts that give rise to behaviour like this.

Stochastic - Gillespie noise

In [ ]:
timeaxis=np.linspace(0, 200, 200)

In [ ]:
# Generate Gillespie noise (raw)
noise_timescale = 2
noise_amp = 50

gill_time_final = 300
gill_num_intervals = 200
gill_noise_time, gill_noise_list = gillespie_noise_raw(
    num_timeseries=1,
    noise_timescale=noise_timescale,
    noise_amp=noise_amp,
    time_final=gill_time_final,
)

print("Gillespie noise generated.")

# Scale Gillespie time axis to fit time axis
for gill_time_element in gill_noise_time:
    gill_time_element -= gill_time_element[0]
    gill_time_element *= timeaxis[-1] / gill_time_element[-1]

# Define arrays
a = 0.996
std = 0.002

a_array = (gill_noise_list[0] * std) + a
b_array=0.00 * gill_noise_time[0] + 0
c_array=0.00 * gill_noise_time[0] + 5

# Simulate
print("FHN simulation starts.")
vs, rs = fitzhugh_nagumo_stochastic(
    timeaxis=gill_noise_time[0],
    a_array=a_array, b_array=b_array, c_array=c_array
)
print("FHN simulation done.")

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(gill_noise_time[0], vs)

plt.subplots(figsize=(20,5))
plt.plot(gill_noise_time[0], a_array)

> Seems to transition between more types of behaviour, but this depends on the dynamic range of $a$, which in turn is derived from the parameters.  I don't think I'm near a regime where noise in the parameters translates to noise in the time series, even if I have 7,000+ time points because I'm taking raw Gillespie noise values.

Bifurcation diagram

Detect whether a time series is noisy (jittery): use signal-to-noise ratio from FFT

In [ ]:
# Define functions

from postprocessor.core.processes.fft import fft, fftParameters

def find_nearest(array, value):
    """find index of nearest value in numpy array"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def get_snr(fft_freqs_df, fft_power_df, cutoff_freq):
    """Get signal-to-noise ratio from a Fourier spectrum

    Get signal-to-noise ratio from a Fourier spectrum. Defines a cut-off
    frequency; frequencies lower than this is considered signal, while
    frequencies higher than this is considered noise. The signal-to-noise
    ratio is defined as the area under the Fourier spectrum to the left of
    the cut-off divided by the area under the Fourier spectrum to the right
    of the cut-off. Follows:

    Parameters
    ----------
    fft_freqs_df : pandas.DataFrame
        DataFrame showing in each row the frequency dimension of each
        Fourier spectrum
    fft_power_df : pandas.DataFrame
        DataFrame showing in each row the periodogram (Fourier spectrum)
    cutoff_freq : float
        cut-off frequency to divide signal and noise
    """
    fft_freqs_array = fft_freqs_df.to_numpy()
    fft_power_array = fft_power_df.to_numpy()
    snr = []
    for rowindex, _ in enumerate(fft_power_array):
        cutoff_freq_nearest = find_nearest(
            fft_freqs_array[rowindex, :], cutoff_freq
        )
        # nans can occur if the origin time series has nans -- skip over these
        if np.isnan(cutoff_freq_nearest):
            snr.append(np.nan)
        else:
            cutoff_colindex = np.where(
                fft_freqs_array[rowindex, :] == cutoff_freq_nearest
            )[0].item()
            area_all = np.trapz(
                y=fft_power_array[rowindex, :], x=fft_freqs_array[rowindex, :]
            )
            area_signal = np.trapz(
                y=fft_power_array[rowindex, 0:cutoff_colindex],
                x=fft_freqs_array[rowindex, 0:cutoff_colindex],
            )
            area_noise = area_all - area_signal
            snr.append(area_signal / area_noise)
    return np.array(snr)

In [ ]:
# Running simulations repeatedly -- inefficient, but works (hopefully)

# Convenience
def fitzhugh_nagumo_a(base_a):
    # Using white noise to speed things up
    a_array=0.002 * np.random.rand(200) + base_a
    b_array=0.00 * np.random.rand(200) + 0
    c_array=0.00 * np.random.rand(200) + 5

    vs, rs = fitzhugh_nagumo_stochastic(
        timeaxis=np.linspace(0, 200, 200),
        a_array=a_array, b_array=b_array, c_array=c_array
    )
    return vs

base_a_list = np.linspace(0.99, 1.00, 50)

snr_values = []
for base_a in base_a_list:
    vs = fitzhugh_nagumo_a(base_a)
    f, p = fft.as_function(pd.DataFrame(vs).T)
    snr = get_snr(f, p, 0.04)
    snr_values.append(snr)
    
snr_values = np.array(snr_values)

In [ ]:
# Or, use Gillespie noise
# THIS ACTUALLY DOESN'T MAKE SENSE BECAUSE THE TIME AXIS IS UNEVEN
# AND YOU CAN'T DO A FOURIER TRANSFORM OF A TIME SERIES WITH AN UNEVEN TIME AXIS

# Generate Gillespie noise -- once, to save time
noise_timescale = 2
noise_amp = 50

gill_time_final = 300
gill_num_intervals = 200
gill_noise_time, gill_noise_list = gillespie_noise_raw(
    num_timeseries=1,
    noise_timescale=noise_timescale,
    noise_amp=noise_amp,
    time_final=gill_time_final,
)

print("Gillespie noise generated.")

# Scale Gillespie time axis to fit time axis
for gill_time_element in gill_noise_time:
    gill_time_element -= gill_time_element[0]
    gill_time_element *= timeaxis[-1] / gill_time_element[-1]

# Convenience
def fitzhugh_nagumo_a(base_a):
    a_array=(gill_noise_list[0] * 0.002) + a
    b_array=0.00 * gill_noise_time[0] + 0
    c_array=0.00 * gill_noise_time[0] + 5

    vs, rs = fitzhugh_nagumo_stochastic(
        timeaxis=gill_noise_time[0],
        a_array=a_array, b_array=b_array, c_array=c_array
    )
    return vs

base_a_list = np.linspace(0.99, 1.00, 10)

snr_values = []
for base_a in base_a_list:
    print(base_a)
    vs = fitzhugh_nagumo_a(base_a)
    f, p = fft.as_function(pd.DataFrame(vs).T)
    snr = get_snr(f, p, 0.04)
    snr_values.append(snr)
    
snr_values = np.array(snr_values)

In [ ]:
plt.plot(base_a_list, snr_values)

In [ ]:
plt.plot(f.T, p.T)

In [ ]:
vs = fitzhugh_nagumo_a(0.996)

plt.subplots()
plt.plot(vs)

f, p = fft.as_function(pd.DataFrame(vs).T)
plt.subplots()
plt.plot(f.T, p.T)

In [ ]:
# 2d
# Running simulations repeatedly -- inefficient, but works (hopefully)

# Convenience
def fitzhugh_nagumo_ab(base_a, base_b):
    # Using white noise to speed things up
    a_array=0.002 * np.random.rand(200) + base_a
    b_array=0.002 * np.random.rand(200) + base_b
    c_array=0.00 * np.random.rand(200) + 5

    vs, rs = fitzhugh_nagumo_stochastic(
        timeaxis=np.linspace(0, 200, 200),
        a_array=a_array, b_array=b_array, c_array=c_array
    )
    return vs

base_a_list = np.linspace(0, 1, 40)
base_b_list = np.linspace(0, 2, 40)

snr_values = np.zeros((len(base_a_list), len(base_b_list)))
fft_peak_height = np.zeros((len(base_a_list), len(base_b_list)))
for i, base_a in enumerate(base_a_list):
    for j, base_b in enumerate(base_b_list):
        vs = fitzhugh_nagumo_ab(base_a, base_b)
        f, p = fft.as_function(pd.DataFrame(vs).T)
        snr = get_snr(f, p, 0.04)
        snr_values[i][j] = snr
        fft_peak_height[i][j] = np.max(p.to_numpy())

In [ ]:
point = (0., 0.3)

plt.subplots()
plt.contourf(base_a_list, base_b_list, snr_values.T)
plt.colorbar()
plt.plot(point[0], point[1], 'ro')
plt.title('SNR (measure noise)')

plt.subplots()
plt.contourf(base_a_list, base_b_list, fft_peak_height.T)
plt.colorbar()
plt.plot(point[0], point[1], 'ro')
plt.title('FFT peak height (evaluate whether oscillatory)')

In [ ]:
vs = fitzhugh_nagumo_ab(point[0], point[1])

plt.subplots()
plt.plot(vs)

plt.subplots()
plt.plot(vs[75:200])

f, p = fft.as_function(pd.DataFrame(vs).T)
plt.subplots()
plt.plot(f.T, p.T)

In [ ]:
np.max(p.to_numpy())